In [2]:
import pandas as pd
import numpy as np
import math

In [7]:
filename = '../data/1M-LdSt2.csv'
df = pd.read_csv(filename)

In [14]:
df = df.replace('NAN', np.nan).dropna()
df = df.astype('int64')

In [17]:
# Build a dataframe with the 64bit values split into 32 bit values
df1 = pd.DataFrame(index=df.index)
df1['pc-l'] = df['pc'] % math.pow(2, 31) # lower 32 bits
df1['ea-l'] = df['effective_address'] % math.pow(2, 31)
df1['ea-u'] = (df['effective_address'] - df1['ea-l'])/math.pow(2, 31) #remove trailing zeros
df1['v0-l'] = df['val0'] % math.pow(2, 31)
df1['v0-u'] = (df['val0'] - df1['v0-l'])/math.pow(2, 31) #remove trailing zeros

In [18]:
# Scale all values in df1 into the interval [0,1]
df1['pc-l'] = df1['pc-l']/df1['pc-l'].max()
df1['ea-l'] = df1['ea-l']/df1['ea-l'].max()
df1['ea-u'] = df1['ea-u']/df1['ea-u'].max()
df1['v0-l'] = df1['v0-l']/df1['v0-l'].max()
df1['v0-u'] = df1['v0-u']/df1['v0-u'].max()

In [20]:
k=4

In [21]:
# Add columns for the previous k loads
for i in range(1, k+1):
    lname = 'v0l-' + str(i)
    df1[lname] = df1['v0-l'].shift(i)
    uname = 'v0u-' + str(i)
    df1[uname] = df1['v0-u'].shift(i)

In [22]:
df

,pc,effective_address,val0,store-1ea,store-1val0,store-2ea,store-2val0
7,6765468,69866272,3407933,281474976692496,281474976692448,281474976692480,281474976692448
8,6765496,8943776,6765512,281474976692496,281474976692448,281474976692480,281474976692448
9,6765512,281474976692496,69866368,281474976692496,281474976692448,281474976692480,281474976692448
10,6765516,281474976692480,9304038,281474976692496,281474976692448,281474976692480,281474976692448
11,6765520,281474976692464,46356912,281474976692496,281474976692448,281474976692480,281474976692448
...,...,...,...,...,...,...,...
45356,8367700,281474976692544,19171360,20767208,20767224,20767200,20767216
45357,8367704,281474976692560,31,20767208,20767224,20767200,20767216
45358,8367708,281474976692528,281474976692608,20767208,20767224,20767200,20767216
45359,4789432,10459048,58217,19171360,4789744,20767208,20767224


In [24]:
df1['s-1l'] = df['store-1val0'] % math.pow(2, 31) # lower 32 bits
df1['s-1u'] = (df['store-1val0'] - df1['s-1l']) / math.pow(2, 31)
df1['s-2l'] = df['store-2val0'] % math.pow(2, 31) # lower 32 bits
df1['s-2u'] = (df['store-2val0'] - df1['s-2l']) / math.pow(2, 31)

In [26]:
df1['s-1l'] = df1['s-1l'] / df1['s-1l'].max()
df1['s-1u'] = df1['s-1u'] / df1['s-1u'].max()
df1['s-2l'] = df1['s-2l'] / df1['s-2l'].max()
df1['s-2u'] = df1['s-2u'] / df1['s-2u'].max()

In [27]:
df1

,pc-l,ea-l,ea-u,v0-l,v0-u,v0l-1,v0u-1,v0l-2,v0u-2,v0l-3,v0u-3,v0l-4,v0u-4,s-1l,s-1u,s-2l,s-2u
7,0.808478,0.032534,0.0,1.586942e-03,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999992,1.0,0.999992,1.0
8,0.808481,0.004165,0.0,3.150437e-03,0.000000,1.586942e-03,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.999992,1.0,0.999992,1.0
9,0.808483,1.000000,1.0,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.000000,NaN,NaN,NaN,NaN,0.999992,1.0,0.999992,1.0
10,0.808484,1.000000,1.0,4.332530e-03,0.000000,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.0,NaN,NaN,0.999992,1.0,0.999992,1.0
11,0.808484,1.000000,1.0,2.158662e-02,0.000000,4.332530e-03,0.000000,3.253406e-02,0.000000,3.150437e-03,0.0,0.001587,0.0,0.999992,1.0,0.999992,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45356,0.999946,1.000000,1.0,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.0,0.009670,0.0,0.009670,0.0,0.009670,0.0
45357,0.999946,1.000000,1.0,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.0,0.009670,0.0,0.009670,0.0,0.009670,0.0
45358,0.999947,1.000000,1.0,9.999916e-01,0.000031,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.0,0.000000,0.0,0.009670,0.0,0.009670,0.0
45359,0.572340,0.004870,0.0,2.710940e-05,0.000000,9.999916e-01,0.000031,1.443550e-08,0.000000,8.927360e-03,0.0,0.021587,0.0,0.002230,0.0,0.009670,0.0


In [28]:
# Reorder the columns so the 'outputs' are the last 2 columns
df1 = df1[["pc-l", "ea-l", "ea-u", "v0l-1", "v0u-1", "v0l-2", "v0u-2", "v0l-3", \
           "v0u-3", "v0l-4", "v0u-4", "s-1l", "s-1u", "s-2l", "s-2u", "v0-l", "v0-u"]]

In [29]:
df1

,pc-l,ea-l,ea-u,v0l-1,v0u-1,v0l-2,v0u-2,v0l-3,v0u-3,v0l-4,v0u-4,s-1l,s-1u,s-2l,s-2u,v0-l,v0-u
7,0.808478,0.032534,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999992,1.0,0.999992,1.0,1.586942e-03,0.000000
8,0.808481,0.004165,0.0,1.586942e-03,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.999992,1.0,0.999992,1.0,3.150437e-03,0.000000
9,0.808483,1.000000,1.0,3.150437e-03,0.000000,1.586942e-03,0.000000,NaN,NaN,NaN,NaN,0.999992,1.0,0.999992,1.0,3.253406e-02,0.000000
10,0.808484,1.000000,1.0,3.253406e-02,0.000000,3.150437e-03,0.000000,1.586942e-03,0.0,NaN,NaN,0.999992,1.0,0.999992,1.0,4.332530e-03,0.000000
11,0.808484,1.000000,1.0,4.332530e-03,0.000000,3.253406e-02,0.000000,3.150437e-03,0.0,0.001587,0.0,0.999992,1.0,0.999992,1.0,2.158662e-02,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45356,0.999946,1.000000,1.0,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.0,0.009670,0.0,0.009670,0.0,0.009670,0.0,8.927360e-03,0.000000
45357,0.999946,1.000000,1.0,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.0,0.009670,0.0,0.009670,0.0,0.009670,0.0,1.443550e-08,0.000000
45358,0.999947,1.000000,1.0,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.0,0.000000,0.0,0.009670,0.0,0.009670,0.0,9.999916e-01,0.000031
45359,0.572340,0.004870,0.0,9.999916e-01,0.000031,1.443550e-08,0.000000,8.927360e-03,0.0,0.021587,0.0,0.002230,0.0,0.009670,0.0,2.710940e-05,0.000000


In [33]:
def prepDataFrame(filename, k=4):
    """ Read in the loads and stores from a trace and 
    """

    # Read in the data and get it ready for computation
    df = pd.read_csv(filename)
    df = df.replace('NAN', np.nan).dropna()
    df = df.astype('int64')

    # Build a dataframe with the 64bit values split into 32 bit values
    df1 = pd.DataFrame(index=df.index)
    df1['pc-l'] = df['pc'] % math.pow(2, 31) # lower 32 bits
    df1['ea-l'] = df['effective_address'] % math.pow(2, 31)
    df1['ea-u'] = (df['effective_address'] - df1['ea-l'])/math.pow(2, 31) #remove trailing zeros
    df1['v0-l'] = df['val0'] % math.pow(2, 31)
    df1['v0-u'] = (df['val0'] - df1['v0-l'])/math.pow(2, 31) #remove trailing zeros

    # Scale all values in df1 into the interval [0,1]
    df1['pc-l'] = df1['pc-l']/df1['pc-l'].max()
    df1['ea-l'] = df1['ea-l']/df1['ea-l'].max()
    df1['ea-u'] = df1['ea-u']/df1['ea-u'].max()
    df1['v0-l'] = df1['v0-l']/df1['v0-l'].max()
    df1['v0-u'] = df1['v0-u']/df1['v0-u'].max()

    # Add columns for the previous k loads
    for i in range(1, k+1):
        lname = 'v0l-' + str(i)
        df1[lname] = df1['v0-l'].shift(i)
        uname = 'v0u-' + str(i)
        df1[uname] = df1['v0-u'].shift(i)

    df1['s-1l'] = df['store-1val0'] % math.pow(2, 31) # lower 32 bits
    df1['s-1u'] = (df['store-1val0'] - df1['s-1l']) / math.pow(2, 31)
    df1['s-2l'] = df['store-2val0'] % math.pow(2, 31) # lower 32 bits
    df1['s-2u'] = (df['store-2val0'] - df1['s-2l']) / math.pow(2, 31)

    df1['s-1l'] = df1['s-1l'] / df1['s-1l'].max()
    df1['s-1u'] = df1['s-1u'] / df1['s-1u'].max()
    df1['s-2l'] = df1['s-2l'] / df1['s-2l'].max()
    df1['s-2u'] = df1['s-2u'] / df1['s-2u'].max()

    # Reorder the columns so the 'outputs' are the last 2 columns
    df1 = df1[["pc-l", "ea-l", "ea-u", "v0l-1", "v0u-1", "v0l-2", "v0u-2", "v0l-3", \
               "v0u-3", "v0l-4", "v0u-4", "s-1l", "s-1u", "s-2l", "s-2u", "v0-l", "v0-u"]]

    return df1.dropna()


In [34]:
prepDataFrame(filename, k=4)

,pc-l,ea-l,ea-u,v0l-1,v0u-1,v0l-2,v0u-2,v0l-3,v0u-3,v0l-4,v0u-4,s-1l,s-1u,s-2l,s-2u,v0-l,v0-u
11,0.808484,1.000000,1.0,4.332530e-03,0.000000,3.253406e-02,0.000000,3.150437e-03,0.000000,0.001587,0.0,0.999992,1.0,0.999992,1.0,2.158662e-02,0.000000
12,0.808485,1.000000,1.0,2.158662e-02,0.000000,4.332530e-03,0.000000,3.253406e-02,0.000000,0.003150,0.0,0.999992,1.0,0.999992,1.0,9.999916e-01,0.000031
13,0.808483,1.000000,1.0,9.999916e-01,0.000031,2.158662e-02,0.000000,4.332530e-03,0.000000,0.032534,0.0,0.999992,1.0,0.999992,1.0,3.253409e-02,0.000000
14,0.808484,1.000000,1.0,3.253409e-02,0.000000,9.999916e-01,0.000031,2.158662e-02,0.000000,0.004333,0.0,0.999992,1.0,0.999992,1.0,4.332596e-03,0.000000
15,0.808484,1.000000,1.0,4.332596e-03,0.000000,3.253409e-02,0.000000,9.999916e-01,0.000031,0.021587,0.0,0.999992,1.0,0.999992,1.0,2.158662e-02,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45356,0.999946,1.000000,1.0,2.158709e-02,0.000000,0.000000e+00,0.000000,9.670496e-03,0.000000,0.009670,0.0,0.009670,0.0,0.009670,0.0,8.927360e-03,0.000000
45357,0.999946,1.000000,1.0,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000e+00,0.000000,0.009670,0.0,0.009670,0.0,0.009670,0.0,1.443550e-08,0.000000
45358,0.999947,1.000000,1.0,1.443550e-08,0.000000,8.927360e-03,0.000000,2.158709e-02,0.000000,0.000000,0.0,0.009670,0.0,0.009670,0.0,9.999916e-01,0.000031
45359,0.572340,0.004870,0.0,9.999916e-01,0.000031,1.443550e-08,0.000000,8.927360e-03,0.000000,0.021587,0.0,0.002230,0.0,0.009670,0.0,2.710940e-05,0.000000


In [3]:
filename = '../data/compute_fp_1-LdSt2.csv'
#df = prepDataFrame(filename)

df = pd.read_csv(filename)
df = df.replace('NAN', np.nan).dropna()

/home/shoemarw/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df

,pc,effective_address,val0,store-1ea,store-1val0,store-2ea,store-2val0
549704,4413620.0,2.814750e+14,1,281474976705216,281474976705216,281474976705280,281474976705232
549705,4413628.0,2.814750e+14,0,281474976705216,281474976705216,281474976705280,281474976705232
549706,4413780.0,2.814750e+14,281474976705232,281474976706648,1,281474976706632,1
549707,4414888.0,2.814750e+14,1,281474976705216,281474976705200,281474976705200,281474976705200
549708,4414900.0,2.814750e+14,0,281474976705216,281474976705200,281474976705200,281474976705200
...,...,...,...,...,...,...,...
8797068,4400648.0,2.814750e+14,47677440,281474976706872,54995,47897416,47677440
8797069,4400656.0,2.814750e+14,54995,47897420,47677440,281474976706872,54995
8797070,4400660.0,2.814750e+14,920683,47897420,47677440,281474976706872,54995
8797071,4400620.0,1.468979e+08,13839762157265453758,281474976706872,54996,47897420,47677440


In [7]:
df['store-1val0'].astype(np.int64)

OverflowError: Python int too large to convert to C long

In [8]:
import sys
sys.maxsize


9223372036854775807

In [9]:
math.log(sys.maxsize, 2)

63.0